In [14]:
from LoadVars import *
from ga import *
from NeuralNet import NeuralNet, train, calculate
from performance import calc_hv
from SaveOutput import save
import matplotlib.pyplot as plt
import torch

In [15]:
#Defining problem
problem = define_problem(problem_name)
pareto_front = problem.pareto_front()
print(f'The benchmark problem: {problem_name.upper()}\n')

The benchmark problem: OSY



In [16]:
#Initial sampling
initial_sampling = define_sampling(initial_sampling_method_name)
print(f'Performing initial sampling: {initial_sampling_method_name.upper()}\n')
InitialData = initial_sampling.do(problem, n_samples=pop_size, pop=None)

Performing initial sampling: REAL_LHS



In [17]:
#Evaluating initial samples (true eval)
InitialEval = problem.evaluate(InitialData, return_values_of=['F'])
InitialEval_G = problem.evaluate(InitialData, return_values_of=['G'])
InitialEval_CV = problem.evaluate(InitialData, return_values_of=['CV'])

if InitialEval_G is not None:
	InitialEval = np.concatenate((InitialEval, InitialEval_G, InitialEval_CV), axis=1)

save('OUTPUT/initial_pop_X.dat', InitialData, header='#Generation 1: X')
save('OUTPUT/initial_pop_FGCV.dat', InitialEval, header='#Generation 1: F, G, CV')
save('OUTPUT/all_pop_X.dat', InitialData, header='Generation 1: X')
save('OUTPUT/all_pop_FGCV.dat', InitialEval, header='Generation 1: F, G, CV')
save('DATA/training/X.dat', InitialData)
save('DATA/training/OUT.dat', InitialEval[:, 0:problem.n_obj+problem.n_constr])

In [18]:
#Initial performance
HV = [0.0]
HV += [calc_hv(InitialEval[:,range(problem.n_obj)], ref=hv_ref)]

In [19]:
#Initial training for neural nets
print('Feeding the training data to the neural net...\n\n')

Model = NeuralNet(D_in=problem.n_var,
				  H=N_Neuron, D=N_Neuron,
				  D_out=problem.n_obj+problem.n_constr)

print('Performing initial training...\n')

TrainedModel = train(problem=problem,
					 model=Model,
				     N_Epoch=N_Epoch,
				     lr=lr,
				     batchrate=batchrate)
print('\nAn initial trained model is obtained!\n')
print('--------------------------------------------------')
TrainedModel_Problem = TrainedModelProblem(problem, TrainedModel)

Feeding the training data to the neural net...


Performing initial training...

N_Epoch = 0 Loss = 24.511734008789062
N_Epoch = 100 Loss = 0.4098881781101227
N_Epoch = 200 Loss = 0.38681069016456604
N_Epoch = 300 Loss = 0.34612932801246643
N_Epoch = 400 Loss = 0.269835501909256
N_Epoch = 500 Loss = 0.17243728041648865
N_Epoch = 600 Loss = 0.08344240486621857
N_Epoch = 700 Loss = 0.043640412390232086
N_Epoch = 800 Loss = 0.02196873165667057

An initial trained model is obtained!

--------------------------------------------------


In [28]:
#Evolutionary computation routines on the Trained Model

selection = define_selection(selection_operator_name)
crossover = define_crossover(crossover_operator_name, prob=prob_c, eta=eta_c)
mutation = define_mutation(mutation_operator_name, eta=eta_m)

#EA settings
EA = EvolutionaryAlgorithm(algorithm_name)
algorithm = EA.setup(pop_size=pop_size,
					 sampling=initial_sampling,
# 					 selection=selection,
					 crossover=crossover,
					 mutation=mutation)

#Stopping criteria
stopping_criteria_def = StoppingCriteria(termination_name)
stopping_criteria = stopping_criteria_def.set_termination(n_gen=n_gen)

#Obtaining optimal solutions on the initial trained model
print(f'Performing optimization on the initial trained model using {algorithm_name.upper()}\n')
optimal_solutions =  do_optimization(TrainedModel_Problem,
									 algorithm, stopping_criteria,
									 verbose=True, seed=1,
									 return_least_infeasible=False)
print('--------------------------------------------------')
print('\nOptimal solutions on the initial trained model is obtained!\n')
print('--------------------------------------------------')

Performing optimization on the initial trained model using NSGA2

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     100 |  3.946895063 |  1.27354E+01 |       1 |            - |            -
    2 |     200 |  3.893940032 |  7.771185580 |       1 |      26.3302 |        ideal
    3 |     300 |  3.614712000 |  5.624607941 |       1 |     9.530151 |        ideal
    4 |     400 |  3.387912214 |  4.503499701 |       1 |    4.6553955 |        ideal
    5 |     500 |  3.387912214 |  3.990491118 |       1 |  0.00000E+00 |            f
    6 |     600 |  3.322402537 |  3.692812963 |       1 |    6.0720215 |        ideal
    7 |     700 |  3.202178717 |  3.519517620 |       1 |     6.963867 |        ideal
    8 |     800 |  3.191092014 |  3.426038094 |       1 |    6.8904076 |        ideal
    9 |     900 |  3.171473324 |  3.363428976 |       1 |      8.01759 |        ideal
   10 |    1000 |  3.158424139 |  3.306114809 |       1 |    2.2176514 |  